In [97]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import requests
from scipy.stats import norm

In [42]:
# current portfolio
# portfolio = pd.read_csv('results/top10portfoliopick.csv')
portfolio = pd.read_csv('/Users/alexvalentine/Documents/GitHub/PortfolioProject/results/top10portfoliopick.csv')

In [43]:
print(portfolio)


  Stock  Monthly Return
0  EBAY        0.139541
1   ACN        0.095700
2   CMG        0.090364
3   FIS        0.081264
4    HD        0.076083
5   MAR        0.068969
6   AMD        0.068306
7   AXP        0.067618
8    GM        0.067295
9   AMT        0.065032


In [106]:

# Black-Scholes
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return price


# add options strategy
def add_options_strategy(portfolio, stock_symbol, option_type, strike_price, expiration_days, volatility):
    stock = portfolio[portfolio['Symbol'] == stock_symbol]
    if stock.empty:
        print(f"stock {stock_symbol} not in current portfolio")
        return portfolio
    
    # Get current stock price using yfinance
    ticker = yf.Ticker(stock_symbol)
    current_price = ticker.history(period='1d')['Close'].iloc[-1]
    
    r = 0.02  
    T = expiration_days / 365
    
    option_price = black_scholes(current_price, strike_price, T, r, volatility, option_type)
    
    # add option to portfolio
    new_row = pd.DataFrame({
        'Symbol': [f"{stock_symbol}_{option_type.upper()}"],
        'Current Price': [current_price],
        'Volatility': [volatility],
        'Strike Price': [strike_price],
        'Expiration Days': [expiration_days],
        'Option Type': [option_type],
        'Type': ['Option'],
        'Price': [option_price],
        'Quantity': [100],  # assume 1 option contract (100 shares)
        'Strike': [strike_price],
        'Expiration': [expiration_days],
        'Underlying': [stock_symbol]
    })
    
    return new_row


In [69]:
# get past one year monthly data of stocks in portfolio using yfinance
import yfinance as yf

stock_symbols = portfolio['Stock'].tolist()

# get past one year monthly data
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=1)

results = pd.DataFrame(columns=['Symbol', 'Current Price', 'Volatility'])

for symbol in stock_symbols:
    historical_data = yf.download(symbol, start=start_date, end=end_date, interval='1mo')
    
    # use adjusted close price
    historical_prices = historical_data['Adj Close']
    
    # calculate current price
    current_price = historical_prices.iloc[-1]
    
    # calculate volatility (standard deviation)
    volatility = np.std(historical_prices.pct_change().dropna()) * np.sqrt(252)  # annualized volatility
    
    new_row = pd.DataFrame({
        'Symbol': [symbol],
        'Current Price': [current_price],
        'Volatility': [volatility]
    })
    results = pd.concat([results, new_row], ignore_index=True)

# display results
print(results)


[*********************100%***********************]  1 of 1 completed
/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_35513/3478731013.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_row], ignore_index=True)
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********

  Symbol  Current Price  Volatility
0   EBAY      63.099998    1.005718
1    ACN     360.799988    1.041537
2    CMG      59.439999    0.991146
3    FIS      89.720001    0.856065
4     HD     398.910004    1.008254
5    MAR     262.119995    0.831746
6    AMD     156.229996    1.706587
7    AXP     267.350006    0.659609
8     GM      52.070000    1.253116
9    AMT     222.800003    1.276067


In [70]:

# define strike price (10% above current price)
results['Strike Price'] = results['Current Price'] * 1.1

# define expiration time (30 days)
results['Expiration Days'] = 30

# define option type (assume we expect price to rise)
results['Option Type'] = 'call'

# output parameters
print(results)

  Symbol  Current Price  Volatility  Strike Price  Expiration Days Option Type
0   EBAY      63.099998    1.005718     69.409998               30        call
1    ACN     360.799988    1.041537    396.879987               30        call
2    CMG      59.439999    0.991146     65.383998               30        call
3    FIS      89.720001    0.856065     98.692001               30        call
4     HD     398.910004    1.008254    438.801004               30        call
5    MAR     262.119995    0.831746    288.331995               30        call
6    AMD     156.229996    1.706587    171.852995               30        call
7    AXP     267.350006    0.659609    294.085007               30        call
8     GM      52.070000    1.253116     57.277000               30        call
9    AMT     222.800003    1.276067    245.080003               30        call


In [22]:
stock_symbols = df['Stock'].tolist()
# historiical option price

In [62]:
# historiical option price
api_key = '5BSPZSJOMAK2A8CA'
# limit 25 calls per day per api key

options_df = pd.DataFrame()
for symbol in stock_symbols:
    url = f'https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS&symbol={symbol}&apikey={api_key}'
    r = requests.get(url)
    option = pd.json_normalize(r.json(), record_path=['data'])
    options_df = pd.concat([options_df, option], ignore_index=True)


In [66]:
options_df.to_csv('options_data.csv', index=False)

In [77]:
display(options_df)

,contractID,symbol,expiration,strike,type,last,mark,bid,bid_size,ask,ask_size,volume,open_interest,date,implied_volatility,delta,gamma,theta,vega,rho
0,EBAY241025C00030000,EBAY,2024-10-25,30.00,call,35.81,33.05,31.35,50,34.75,50,0,1,2024-10-25,4.20963,1.00000,0.00000,-0.01449,0.00000,0.00082
1,EBAY241025P00030000,EBAY,2024-10-25,30.00,put,0.00,0.01,0.00,0,0.52,3,0,0,2024-10-25,5.14166,-0.00188,0.00035,-0.18561,0.00020,-0.00000
2,EBAY241025C00035000,EBAY,2024-10-25,35.00,call,0.00,28.20,26.65,50,29.75,50,0,0,2024-10-25,5.45918,0.98630,0.00194,-1.16867,0.00116,0.00093
3,EBAY241025P00035000,EBAY,2024-10-25,35.00,put,0.00,0.01,0.00,0,0.52,4,0,0,2024-10-25,4.13802,-0.00233,0.00053,-0.18137,0.00024,-0.00000
4,EBAY241025C00040000,EBAY,2024-10-25,40.00,call,0.00,23.02,22.35,50,23.70,51,0,0,2024-10-25,2.93784,1.00000,0.00000,-0.01932,0.00000,0.00110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14347,AMT270115P00330000,AMT,2027-01-15,330.00,put,0.00,107.50,105.00,1,110.00,10,0,0,2024-10-25,0.42485,-0.55310,0.00280,-0.01402,1.31398,-5.13306
14348,AMT270115C00340000,AMT,2027-01-15,340.00,call,0.00,5.45,3.50,20,7.40,12,0,0,2024-10-25,0.19635,0.17621,0.00397,-0.05430,0.86037,0.75207
14349,AMT270115P00340000,AMT,2027-01-15,340.00,put,0.00,117.00,114.50,10,119.50,10,0,0,2024-10-25,0.44345,-0.55799,0.00268,-0.01418,1.31170,-5.36836
14350,AMT270115C00350000,AMT,2027-01-15,350.00,call,5.03,4.53,2.55,12,6.50,17,0,3,2024-10-25,0.19620,0.15153,0.00360,-0.04852,0.78006,0.65031


In [107]:
options_portfolio = pd.DataFrame()

# add_options_strategy(portfolio, stock_symbol, option_type, strike_price, expiration_days, volatility):
for _,row in results.iterrows():
    op = add_options_strategy(results, row[0], row[5], row[3], row[4], row[2])
    options_portfolio = pd.concat([options_portfolio, op], ignore_index=True)

display(options_portfolio)


/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_35513/2410265079.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  op = add_options_strategy(results, row[0], row[5], row[3], row[4], row[2])
/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_35513/2410265079.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  op = add_options_strategy(results, row[0], row[5], row[3], row[4], row[2])
/var/folders/n3/cwx8131570q5ljqcv1f22n6c0000gn/T/ipykernel_35513/2410265079.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as 

,Symbol,Current Price,Volatility,Strike Price,Expiration Days,Option Type,Type,Price,Quantity,Strike,Expiration,Underlying
0,EBAY_CALL,63.099998,1.005718,69.409998,30,call,Option,4.884099,100,69.409998,30,EBAY
1,ACN_CALL,360.799988,1.041537,396.879987,30,call,Option,29.383085,100,396.879987,30,ACN
2,CMG_CALL,59.439999,0.991146,65.383998,30,call,Option,4.503402,100,65.383998,30,CMG
3,FIS_CALL,89.720001,0.856065,98.692001,30,call,Option,5.445391,100,98.692001,30,FIS
4,HD_CALL,398.910004,1.008254,438.801004,30,call,Option,30.990442,100,438.801004,30,HD
5,MAR_CALL,262.119995,0.831746,288.331995,30,call,Option,15.205230,100,288.331995,30,MAR
6,AMD_CALL,156.229996,1.706587,171.852995,30,call,Option,24.571462,100,171.852995,30,AMD
7,AXP_CALL,267.350006,0.659609,294.085007,30,call,Option,10.532275,100,294.085007,30,AXP
8,GM_CALL,52.070000,1.253116,57.277000,30,call,Option,5.490988,100,57.277000,30,GM
9,AMT_CALL,222.800003,1.276067,245.080003,30,call,Option,24.078255,100,245.080003,30,AMT


In [114]:
filtered_options = pd.DataFrame()

for _, row in options_portfolio.iterrows():
    # Convert expiration days to date
    expiry_date = pd.Timestamp.today() + pd.Timedelta(days=row['Expiration'])
    expiry_date = expiry_date.strftime('%Y-%m-%d')
    
    # Convert strike prices to float for comparison and round up to nearest 10
    options_strike = pd.to_numeric(options_df['strike'], errors='coerce')
    portfolio_strike = float(np.ceil(float(row['Strike']) / 10) * 10)
    
    mask = (options_df['symbol'] == row['Underlying']) & \
           (abs(options_strike - portfolio_strike) < 0.01) & \
           (options_df['type'].str.lower() == row['Option Type'].lower()) & \
           (abs(pd.to_datetime(options_df['expiration']) - pd.to_datetime(expiry_date)) <= pd.Timedelta(days=7))
    
    matching_options = options_df[mask].copy()
    if not matching_options.empty:
        # Add portfolio information
        matching_options['portfolio_price'] = row['Price'] 
        matching_options['portfolio_quantity'] = row['Quantity']
        matching_options['portfolio_strike'] = portfolio_strike  # Use rounded strike price
        matching_options['portfolio_expiration_days'] = row['Expiration']
        matching_options['portfolio_volatility'] = row['Volatility']
        matching_options['portfolio_current_price'] = row['Current Price']
        
        filtered_options = pd.concat([filtered_options, matching_options], ignore_index=True)

display(filtered_options)


,contractID,symbol,expiration,strike,type,last,mark,bid,bid_size,ask,...,gamma,theta,vega,rho,portfolio_price,portfolio_quantity,portfolio_strike,portfolio_expiration_days,portfolio_volatility,portfolio_current_price
0,EBAY241122C00070000,EBAY,2024-11-22,70.00,call,0.69,1.15,0.39,19,1.91,...,0.03758,-0.17934,0.05510,0.01104,4.884099,100,70.0,30,1.005718,63.099998
1,EBAY241129C00070000,EBAY,2024-11-29,70.00,call,0.58,0.55,0.46,31,0.64,...,0.04059,-0.08810,0.04971,0.00984,4.884099,100,70.0,30,1.005718,63.099998
2,ACN241122C00400000,ACN,2024-11-22,400.00,call,0.85,0.90,0.10,50,1.70,...,0.00600,-0.25934,0.15027,0.02179,29.383085,100,400.0,30,1.041537,360.799988
3,ACN241129C00400000,ACN,2024-11-29,400.00,call,0.75,0.38,0.20,53,0.55,...,0.00462,-0.11084,0.10683,0.01538,29.383085,100,400.0,30,1.041537,360.799988
4,CMG241122C00070000,CMG,2024-11-22,70.00,call,0.40,0.35,0.30,575,0.40,...,0.02522,-0.08934,0.03011,0.00456,4.503402,100,70.0,30,0.991146,59.439999
5,CMG241129C00070000,CMG,2024-11-29,70.00,call,0.38,0.42,0.35,22,0.50,...,0.02618,-0.08186,0.03658,0.00637,4.503402,100,70.0,30,0.991146,59.439999
6,FIS241122C00100000,FIS,2024-11-22,100.00,call,0.00,0.38,0.30,12,0.45,...,0.02535,-0.09530,0.04668,0.00727,5.445391,100,100.0,30,0.856065,89.720001
7,FIS241129C00100000,FIS,2024-11-29,100.00,call,0.00,0.42,0.35,12,0.50,...,0.02601,-0.08266,0.05472,0.00970,5.445391,100,100.0,30,0.856065,89.720001
8,HD241122C00440000,HD,2024-11-22,440.00,call,1.28,1.19,1.02,41,1.35,...,0.00602,-0.32178,0.18524,0.02784,30.990442,100,440.0,30,1.008254,398.910004
9,HD241129C00440000,HD,2024-11-29,440.00,call,1.61,1.52,1.29,31,1.76,...,0.00638,-0.30958,0.23226,0.04062,30.990442,100,440.0,30,1.008254,398.910004


In [115]:
# save updated portfolio
filtered_options.to_csv('results/options_pick.csv', index=False)